In [1]:
%load_ext watermark
%watermark

2020-09-10T12:49:09-05:00

CPython 3.7.6
IPython 7.13.0

compiler   : GCC 7.3.0
system     : Linux
release    : 5.4.0-47-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit


In [2]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import cross_validate

In [3]:
vehiculos = pd.read_csv("vehiculos_procesado.csv")
datos_entrenamiento = vehiculos[["desplazamiento", "cilindros", "consumo"]]

objetivo = vehiculos["co2"]

In [4]:
datos_entrenamiento.head()

,desplazamiento,cilindros,consumo
0,2.5,4.0,17
1,4.2,6.0,13
2,2.5,4.0,16
3,4.2,6.0,13
4,3.8,6.0,16


In [5]:
from sklearn.linear_model import (LinearRegression, Lasso, Ridge, ElasticNet)

In [7]:
modelo_ols = LinearRegression()
modelo_ols.fit(X=datos_entrenamiento, y=objetivo)

modelo_ols.coef_

array([ 11.76787991,   1.23791071, -19.80355606])

In [10]:
def norma_l1(coeficientes):
    return np.abs(coeficientes).sum()

def norma_l2(coeficientes):
    return np.sqrt(np.power(coeficientes,2).sum())

print(norma_l1(modelo_ols.coef_))
print(norma_l2(modelo_ols.coef_))

32.809346680206346
23.069379124496923


Estas mismas funciones se pueden hacer con numpy

In [12]:
def norma_l1(coeficientes):
    return np.linalg.norm(coeficientes, ord=1)

def norma_l2(coeficientes):
    return np.linalg.norm(coeficientes, ord=2)

print(norma_l1(modelo_ols.coef_))
print(norma_l2(modelo_ols.coef_))

32.809346680206346
23.069379124496923


## ¿cómo regularizar las complejidades de los modelos?

In [13]:
from sklearn.preprocessing import PolynomialFeatures

In [14]:
PolynomialFeatures?

In [15]:
transformador_polinomial = PolynomialFeatures(5)

In [16]:
transformador_polinomial.fit(datos_entrenamiento)

PolynomialFeatures(degree=5, include_bias=True, interaction_only=False,
                   order='C')

In [17]:
variables_polinomiales = transformador_polinomial.transform(datos_entrenamiento)

In [18]:
variables_polinomiales.shape

(35539, 56)

In [19]:
datos_entrenamiento.loc[0]

desplazamiento     2.5
cilindros          4.0
consumo           17.0
Name: 0, dtype: float64

In [22]:
variables_polinomiales[0]

array([1.000000e+00, 2.500000e+00, 4.000000e+00, 1.700000e+01,
       6.250000e+00, 1.000000e+01, 4.250000e+01, 1.600000e+01,
       6.800000e+01, 2.890000e+02, 1.562500e+01, 2.500000e+01,
       1.062500e+02, 4.000000e+01, 1.700000e+02, 7.225000e+02,
       6.400000e+01, 2.720000e+02, 1.156000e+03, 4.913000e+03,
       3.906250e+01, 6.250000e+01, 2.656250e+02, 1.000000e+02,
       4.250000e+02, 1.806250e+03, 1.600000e+02, 6.800000e+02,
       2.890000e+03, 1.228250e+04, 2.560000e+02, 1.088000e+03,
       4.624000e+03, 1.965200e+04, 8.352100e+04, 9.765625e+01,
       1.562500e+02, 6.640625e+02, 2.500000e+02, 1.062500e+03,
       4.515625e+03, 4.000000e+02, 1.700000e+03, 7.225000e+03,
       3.070625e+04, 6.400000e+02, 2.720000e+03, 1.156000e+04,
       4.913000e+04, 2.088025e+05, 1.024000e+03, 4.352000e+03,
       1.849600e+04, 7.860800e+04, 3.340840e+05, 1.419857e+06])

Las transformaciones tiene todas las posibles transformaciones de cada record para el grado que le dimos. En este caso es grado 5 por tanto tenemos todas las posibles combinaciones existentes para las tres variables hasta grado 5

In [23]:
variables_polinomiales = PolynomialFeatures(5).fit_transform(datos_entrenamiento)

In [24]:
variables_polinomiales.shape

(35539, 56)

## Tipos de regularización

In [25]:
modelo_ols = LinearRegression()
modelo_ols.fit(variables_polinomiales, objetivo)
print(modelo_ols.coef_)
Resultados = {}
Resultados["ols"] = {
    "norma_l1":norma_l1(modelo_ols.coef_),
    "norma_l2":norma_l2(modelo_ols.coef_)
}

[-1.33584390e-04 -2.16202010e+03 -4.74743030e+02 -9.33379435e+02
  2.06179369e+03 -1.98652141e+03  4.07183962e+02  6.98188921e+02
 -8.60206781e+00  5.68565590e+01 -5.04043555e+02  5.97572364e+02
 -2.39915727e+02 -1.93155159e+02  2.06936510e+02 -2.46367000e+01
 -7.53120028e+00 -6.16587971e+01  2.77167161e+00 -1.72213966e+00
  3.22404684e+01 -2.32860787e+01  3.67724866e+01 -1.28564530e+01
 -4.24157144e+01  9.08423003e+00  1.17166991e+01  1.52755499e+01
 -7.40121866e+00  6.04111964e-01 -9.27527798e-01 -8.19881992e-01
  1.96304324e+00 -8.55528661e-02  2.44388052e-02 -2.44193545e+00
  6.17851513e+00 -1.41371913e+00 -7.50158521e+00  1.59153882e+00
 -6.92569800e-01  5.18941968e+00 -3.95368961e-01  8.24178212e-01
 -1.15475754e-01 -1.85224537e+00 -2.10933026e-02 -3.68950000e-01
  9.99115991e-02 -5.66794753e-03  2.41619837e-01 -2.05795981e-02
  5.80537285e-02 -2.77634650e-02  1.11091110e-03 -1.26148033e-04]


### Modelo regularización L1 (Lasso) con variables polinomiales

In [26]:
Lasso?

In [27]:
modelo_l1 = Lasso(alpha=1.0, tol=0.01, max_iter=5000)
modelo_l1.fit(variables_polinomiales, objetivo)
print(modelo_l1.coef_)
Resultados["regularizacion l1"] = {
    "norma_l1":norma_l1(modelo_l1.coef_),
    "norma_l2":norma_l2(modelo_l1.coef_)
}

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -3.30237023e+01
  5.12409278e+00  1.44708364e+00 -1.47969971e+00  6.52491862e-01
 -5.04276126e-01  5.26431228e-02 -1.48062992e-01 -0.00000000e+00
 -1.32137016e-01  1.02289253e-01 -9.17626300e-02 -1.30192905e-02
  6.15573125e-02 -3.01413839e-02  2.31053922e-02  5.17315962e-03
  1.91792946e-02 -1.51220456e-03 -1.88077866e-03 -1.09782669e-03
 -6.50140180e-03 -2.91026927e-04  1.73571147e-03 -3.33999083e-03
 -1.64104930e-03  1.59784747e-03  1.31929178e-03  1.69629743e-04
 -1.64562586e-03  2.03369130e-04  1.99415033e-05 -2.81072727e-03
  7.11467485e-04  2.46531750e-03 -2.20515928e-03  2.38235311e-04
  6.38712931e-04 -1.09015174e-03 -3.89575283e-04 -8.09552788e-05
  2.70933679e-04 -1.56154368e-04 -1.88340282e-04 -2.27097112e-04
  3.81560605e-05  2.29005451e-06 -5.56615246e-05  1.44114807e-04
  4.13806087e-06  5.51970343e-05  7.62940670e-07 -1.14293565e-06]


### Modelo regularización L2 (Ridge) con variables polinomiales

In [29]:
modelo_l2 = Ridge(alpha=1.0, tol=0.01, max_iter=5000)
modelo_l2.fit(variables_polinomiales, objetivo)
print(modelo_l2.coef_)
Resultados["regularizacion l2"] = {
    "norma_l1":norma_l1(modelo_l2.coef_),
    "norma_l2":norma_l2(modelo_l2.coef_)
}

[ 0.00000000e+00  1.51698521e+01  1.42669293e+00 -9.86970220e+00
  4.84913163e+01  3.40032740e+01  4.08631833e+01  1.87346358e+01
 -3.38488282e+01 -4.32040765e+00 -2.38609505e+01  2.49354657e+01
 -1.68732765e+01 -1.27890316e+01 -6.76534935e+00 -2.53940935e+00
 -4.79876232e+00  6.20820540e+00  2.06730060e+00  2.74911647e-01
  1.53993282e+01 -3.67059200e+01  3.58977238e+00  3.45230729e+01
 -4.39077806e+00  9.62582587e-01 -1.53211869e+01  4.03172287e+00
 -3.81042060e-02  4.61051627e-02  3.15260522e+00 -1.12415559e+00
 -2.48531649e-01 -3.05585035e-02 -7.73558072e-03 -2.33699200e+00
  6.80585060e+00 -8.69787063e-01 -7.95192845e+00  2.20034108e+00
 -1.37716313e-01  4.60274243e+00 -2.17522655e+00  2.25506383e-01
 -1.86521190e-02 -1.28795267e+00  9.40255381e-01 -1.93305613e-01
  1.34231942e-02 -4.68387292e-04  1.24947157e-01 -1.58118637e-01
  5.90745800e-02 -2.26336051e-03  2.49788586e-04  8.13236460e-05]


/home/buitrago/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=4.79744e-19): result may not be accurate.
  overwrite_a=True).T


### Modelo de regularización ElasticNet

In [31]:
modelo_elasticnet = ElasticNet(l1_ratio=0.5, tol=0.01, max_iter=5000)
modelo_elasticnet.fit(variables_polinomiales, objetivo)
print(modelo_elasticnet.coef_)
Resultados["regularizacion elasticnet"] = {
    "norma_l1": norma_l1(modelo_elasticnet.coef_),
    "norma_l2": norma_l2(modelo_elasticnet.coef_)
}

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.33310440e+00
  4.89645007e+00  6.09439149e+00 -1.31816563e+00  2.37723778e+00
 -3.10050434e+00 -6.41934303e-01 -7.84374220e-01 -9.59081564e-02
 -7.18954629e-02  7.19478059e-03 -4.57252233e-02 -6.37787774e-02
  1.22426563e-02  3.46550752e-02  4.26955780e-02  2.66673958e-03
  1.39812144e-02 -8.09891082e-03  8.76409316e-03 -1.69690074e-02
 -2.05177215e-03  1.54020194e-03 -6.29758134e-03 -2.92855315e-03
 -6.86058449e-04  1.42133650e-03 -9.92879254e-04  1.28704557e-04
  5.03220762e-04  6.61024458e-04  5.80200778e-05  3.41020839e-03
  2.27563082e-03  2.71125911e-03 -1.90350591e-03 -7.82250851e-05
  6.56665428e-04 -7.34759017e-04 -9.12167373e-04 -2.43120629e-04
  1.60164558e-04  3.89147755e-04 -5.45731423e-04 -3.49391006e-04
  3.45240438e-05  3.46406797e-05  2.59823852e-04 -6.35175029e-05
 -9.14762151e-05  5.08095098e-05  1.14384625e-05  7.36969809e-07]


In [32]:
Resultados

{'ols': {'norma_l1': 10853.748016063273, 'norma_l2': 3922.1947363276686},
 'regularizacion l1': {'norma_l1': 42.945147280023775,
  'norma_l2': 33.49405578377987},
 'regularizacion l2': {'norma_l1': 457.5175984116758,
  'norma_l2': 109.5482019243697},
 'regularizacion elasticnet': {'norma_l1': 21.002924203163793,
  'norma_l2': 8.997096294952177}}

In [33]:
pd.set_option("display.float_format", lambda x: str(round(x,10)))

In [34]:
resultados_df = pd.DataFrame(Resultados).T
l1_ols = resultados_df.loc["ols","norma_l1"]
l2_ols = resultados_df.loc["ols","norma_l2"]

resultados_df["pct_reduccion_l1"] = 1-resultados_df.norma_l1 / l1_ols
resultados_df["pct_reduccion_l2"] = 1-resultados_df.norma_l2 / l2_ols

resultados_df

,norma_l1,norma_l2,pct_reduccion_l1,pct_reduccion_l2
ols,10853.7480160633,3922.1947363277,0.0,0.0
regularizacion l1,42.94514728,33.4940557838,0.9960432887,0.9914603792
regularizacion l2,457.5175984117,109.5482019244,0.9578470407,0.9720696678
regularizacion elasticnet,21.0029242032,8.997096295,0.9980649151,0.9977061067


La tabla nos indica cuanta complejidad han reducido las diferentes regularizaciones